# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [ ]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
# data = {
#     'table': ['users', 'orders', 'products', 'sales'],
#     'definition': ['Contains user information', 'Contains order details', 
#                    'Contains product information', 'Contains sales records'],
# }
data = {
    'table': ['users', 'orders'],
    'definition': ['Contains user information', 'Contains order details', 
                   ],
}
df = pd.DataFrame(data)
print(df)

    table                 definition
0   users  Contains user information
1  orders     Contains order details


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [14]:
print(text_tables)

employees: Employee information, name...
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level


In [6]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [8]:
#Creating the prompt, with the user questions and the tables definitions.
# Replace the question below with your actual user query
pqt1 = prompt_question_tables.format(
	tables=text_tables,
	question="Which users have placed orders?"
)

In [9]:
print(return_OAI(pqt1))

```json
{
    "tables": ["users", "orders"]
}
```


In [11]:
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="List all orders placed by users named Alice."
)

In [12]:
print(return_OAI(pqt3))

```json
{
    "tables": ["users", "orders"]
}
```


In [13]:

prompt_few_shots_question_tables = """
Here are some examples of how to pick tables from definitions:

Example 1  
Tables:  
users: Contains user information  
orders: Contains order details  
JSON answer: ["users", "orders"]

Example 2  
Tables:  
products: Contains product information  
sales: Contains sales records  
JSON answer: ["products", "sales"]

Now, given these tables:
{tables}

Which tables would you need in a SQL query to answer:
"{question}"
Respond with only a JSON array of table names.
"""

In [14]:
prompt_cot_question_tables = """
You are a SQL expert. Think step by step and explain your reasoning before giving the final answer.

Tables and definitions:
{tables}

User’s question:
{question}

First, list each table’s role. Then decide which are needed. Finally, output a JSON array of the chosen table names.
"""

In [22]:
prompt_improved_question_tables = """
You are an AI assistant who strictly outputs in valid JSON.  
Task: Identify the minimal set of tables required for a SQL query.  
Input format:
  {tables}

Question:
  "{question}"

Requirements:
  1. Analyze definitions only.
  2. Do NOT include any explanations or extra fields.
  3. Return exactly:  
     {{"tables": [<table1>, <table2>, …]}}

Begin.
"""

In [16]:
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question="List all orders placed by users named Alice."
)
print(return_OAI(pqt3))

```json
{
    "tables": ["users", "orders"]
}
```


In [17]:
pqt3 = prompt_few_shots_question_tables.format(
    tables=text_tables,
    question="List all orders placed by users named Alice."
)
print(return_OAI(pqt3))

["users", "orders"]


In [18]:
pqt3 = prompt_cot_question_tables.format(
    tables=text_tables,
    question="List all orders placed by users named Alice."
)
print(return_OAI(pqt3))

First, let's identify the role of each table:
- users: Contains user information such as names
- orders: Contains order details including the user who placed the order

Since we are looking for orders placed by users named Alice, we will need information from both the users and orders tables. We need to join these two tables on the user_id column to retrieve the orders placed by users named Alice.

Therefore, the SQL query will involve the users and orders tables.

JSON array of the chosen table names:
```json
["users", "orders"]
```


In [23]:
pqt3 = prompt_improved_question_tables.format(
    tables=text_tables,
    question="List all orders placed by users named Alice."
)
print(return_OAI(pqt3))


{
  "tables": ["users", "orders"]
}


In [24]:
pqt4 = prompt_question_tables.format(
    tables=text_tables,
    question="Show top spenders."
)
print(return_OAI(pqt4))

```json
{
    "tables": {
        "users": "Contains user information",
        "orders": "Contains order details"
    }
}
```


In [25]:
pqt4 = prompt_few_shots_question_tables.format(
    tables=text_tables,
    question="Show top spenders."
)
print(return_OAI(pqt4))

["users", "orders"]


In [26]:
pqt4 = prompt_cot_question_tables.format(
    tables=text_tables,
    question="Show top spenders."
)
print(return_OAI(pqt4))

First, let's identify the role of each table:
- users: Contains user information such as user id, name, email, etc.
- orders: Contains order details such as order id, user id, total amount, order date, etc.

To show the top spenders, we need information about users and their total spending from the orders table. We will need to join the users and orders tables to calculate the total spending for each user.

Therefore, we need the following tables for this query:
- users
- orders

Output JSON array of the chosen table names:
```json
["users", "orders"]
```


In [ ]:
pqt4 = prompt_improved_question_tables.format(
    tables=text_tables,
    question="Show top spenders."
)
print(return_OAI(pqt4))

{
	"tables": ["users", "orders"]
}


In [28]:
pqt5 = prompt_question_tables.format(
    tables=text_tables,
    question="Which countries have no users?"
)
print(return_OAI(pqt5))

```json
{
    "tables": ["users"]
}
```


In [29]:
pqt5 = prompt_few_shots_question_tables.format(
    tables=text_tables,
    question="Which countries have no users?"
)
print(return_OAI(pqt5))

["users"]


In [30]:
pqt5 = prompt_cot_question_tables.format(

    tables=text_tables,
    question="Which countries have no users?"
)
print(return_OAI(pqt5))

First, let's understand the role of each table:
- users: Contains information about users, including their country.
- orders: Contains details about orders, but does not directly provide information about countries.

Since we are interested in finding countries with no users, we only need the `users` table for this query.

Therefore, the chosen table for this question is:
```json
["users"]
```


In [31]:
pqt5 = prompt_improved_question_tables.format(
    tables=text_tables,
    question="Which countries have no users?"
)
print(return_OAI(pqt5))

{
    "tables": ["users"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

LLM is dependent on on prompt used to map user questions to the correct SQL tables. Zero-Shot can serve as a quick baseline, Few-Shot mirrors the examples. Chain-of-Thought explains the process prior giving the answer, but it looks too long for production. Optimized/combined style prompts deliver more reliable answers.